In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from slugify import slugify

In [19]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [20]:
df = pd.read_csv('csv/localities.csv', encoding='utf-8', delimiter=';')
image = pd.read_csv('out/image.csv', encoding='utf-8')

In [21]:
newDfC = pd.DataFrame(data = df['Continent'].unique())

In [22]:
columns = ['id', 'createdDate', 'updatedDate', 'latitude', 'longitude', 'internalNote', \
          'isPopular', 'order', 'statusId', 'canShowAds', 'isActive', 'slug', \
          'mainImageId', 'coverImageId', 'type', 'countryContinentId', 'regionCountryId', \
          'regionContinentId']

In [23]:
dfCon = newDfC

In [24]:
dfCon = pd.concat([dfCon, pd.DataFrame(columns = columns)])

In [25]:
dfCon.rename(columns = {0:'name'}, inplace = True)

In [26]:
def get_photo_name(value):
    try:
        return df[:][df['Continent'] == str(value)]['Photo name'].values[0]
    except:
        return ''

In [27]:
dfCon['id'] = dfCon.index + 1
dfCon['order'] = ''
dfCon['createdDate'] = dt_string
dfCon['updatedDate'] = dt_string
dfCon['internalNote'] = ''
dfCon['isPopular'] = False
dfCon['statusId'] = 7
dfCon['canShowAds'] = False
dfCon['isActive'] = True
dfCon['type'] = 'Continent'
dfCon['photoName'] = dfCon['name'].apply(get_photo_name)

In [28]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="trynego")

In [29]:
def get_lat(value):
    try:
        return geolocator.geocode(str(value))[1][0]
    except:
        return None

In [30]:
def get_lot(value):
    try:
        return geolocator.geocode(str(value))[1][1]
    except:
        return None

In [34]:
def set_slug_id():
    try:
        for index, row in dfCon.iterrows():
            dfCon.loc[index, ['slug']] = [slugify(row['name'])]
    except:
        pass
        
set_slug_id()

In [36]:
dfCon['latitude'] = dfCon['name'].apply(get_lat)
dfCon['longitude'] = dfCon['name'].apply(get_lot)

In [37]:
dfCon = dfCon[:][dfCon['name'].notnull()]

In [38]:
newDfCountries = pd.DataFrame(data = df['Country'].unique())

In [39]:
def get_photo_name_c(value):
    try:
        return df[:][(df['Country'] == str(value)) & (df['Region'] == '-')]['Photo name'].values[0]
    except:
        return ''

In [40]:
dfCountry = pd.concat([newDfCountries, pd.DataFrame(columns = columns)])

In [41]:
dfCountry.rename(columns = {0:'name'}, inplace = True)

In [43]:
def set_slug_id():
    try:
        for index, row in dfCountry.iterrows():
            dfCountry.loc[index, ['slug']] = [slugify(row['name'])]
    except:
        pass
        
set_slug_id()

In [44]:
dfCountry['id'] = dfCountry.index + dfCon.shape[0]+1
dfCountry['order'] = ''
dfCountry['createdDate'] = dt_string
dfCountry['updatedDate'] = dt_string
dfCountry['internalNote'] = ''
dfCountry['isPopular'] = False
dfCountry['statusId'] = 7
dfCountry['canShowAds'] = False
dfCountry['isActive'] = True
dfCountry['type'] = 'Country'
dfCountry['photoName'] = dfCountry['name'].apply(get_photo_name_c)

In [45]:
dfCountry['latitude'] = dfCountry['name'].apply(get_lat)
dfCountry['longitude'] = dfCountry['name'].apply(get_lot)

In [46]:
def set_continent_id(value):
    try:
        return dfCon[:][dfCon['name'] == df[:][df['Country'] == str(value)]['Continent'].values[0]]['id'].values[0]
    except:
        return ''

In [47]:
def set_image_id_c(value):
    try:
        return image[:][image['name'].str.lower() == 'region_' + str(value).lower().replace(" ", "_")]['id'].values[0]
    except:
        return ''

In [48]:
dfCountry['countryContinentId'] = dfCountry['name'].apply(set_continent_id)

In [49]:
dfCountry['coverImageId'] = dfCountry['name'].apply(set_image_id_c)

In [50]:
dfCountry = dfCountry[:][dfCountry['name'].notnull()]

In [51]:
newDfR = pd.DataFrame(data = df[["Region"]])

In [52]:
dfReg = pd.concat([newDfR, pd.DataFrame(columns = columns)])

In [53]:
dfReg.rename(columns = {'Region':'name'}, inplace = True)

In [56]:
def set_slug_id():
    try:
        for index, row in dfReg.iterrows():
            dfReg.loc[index, ['slug']] = [slugify(row['name'])]
    except:
        pass
        
set_slug_id()

In [57]:
def get_photo_name_r(value):
    try:
        return df[:][df['Region'] == str(value)]['Photo name'].values[0]
    except:
        return ''

In [58]:
temp = (dfCon.shape[0]+1) + (dfCountry.shape[0]+1)
dfReg['id'] = dfReg.index + temp 
dfReg['order'] = ''
dfReg['createdDate'] = dt_string
dfReg['updatedDate'] = dt_string
dfReg['internalNote'] = ''
dfReg['isPopular'] = False
dfReg['statusId'] = 7
dfReg['canShowAds'] = False
dfReg['isActive'] = True
dfReg['type'] = 'Region'
dfReg['photoName'] = dfReg['name'].apply(get_photo_name_r)

In [59]:
dfReg['latitude'] = dfReg['name'].apply(get_lat)
dfReg['longitude'] = dfReg['name'].apply(get_lot)

In [60]:
def set_continent_reg_id(value):
    try:
        return dfCon[:][dfCon['name'] == df[:][df['Region'] == str(value)]['Continent'].values[0]]['id'].values[0]
    except:
        return ''

In [61]:
def set_region_reg_id(value):
    try:
        return dfCountry[:][dfCountry['name'] == df[:][df['Region'] == str(value)]['Country'].values[0]]['id'].values[0]
    except:
        return ''

In [62]:
dfReg['regionContinentId'] = dfReg['name'].apply(set_continent_reg_id)

In [63]:
dfReg['regionCountryId'] = dfReg['name'].apply(set_region_reg_id)

In [64]:
dfReg = dfReg[:][dfReg['name'].notnull()]

In [65]:
def set_image_id(value):
    try:
        return image[:][image['name'].str.lower() == 'region_' + str(value).lower().replace(" ", "_")]['id'].values[0]
    except:
        return ''

dfReg['coverImageId'] = dfReg['photoName'].apply(set_image_id)

In [66]:
def set_image_id_c(value):
    try:
        return image[:][image['name'].str.lower() == 'region_' + str(value).lower().replace(" ", "_")]['id'].values[0]
    except:
        return ''

dfCountry['coverImageId'] = dfCountry['photoName'].apply(set_image_id_c)

In [67]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [68]:
newDf = pd.concat([dfCon, dfCountry, dfReg])

In [69]:
newDf = newDf[:][(newDf['name'] != '') & (newDf['name'] != '-') & (newDf['name'].notnull())]

In [70]:
newDf['id'] = [i for i in range(1, newDf.shape[0] + 1)]

In [71]:
newDf['order'] = newDf.index + 1

In [72]:
newDf['latitude'] = newDf['latitude'].fillna(1)
newDf['longitude'] = newDf['longitude'].fillna(1)

In [73]:
newDf.to_csv('out/localities.csv', index=False, encoding='utf-8')

In [74]:
newDf.shape

(262, 20)

In [75]:
newDfCopy = newDf

In [76]:
def remove_number(value):
    return ''.join([i for i in str(value) if not i.isdigit()])

In [77]:
newDfCopy['name'] = newDfCopy['name'].apply(remove_number)

In [78]:
newDfCopy.to_csv('out/localities_main.csv', index=False, encoding='utf-8')